# IMPORTS

In [2]:
from collections import *
from functools import *
from itertools import *
import operator
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib
import unittest
import copy
from dataclasses import dataclass, field
import sympy
import networkx as nx
from pyvis.network import Network
from typing import *
from shapely.geometry import Polygon
from enum import StrEnum, Enum
from abc import ABC, abstractmethod

In [36]:
from itertools import product

class PatrolGrid:
    DIR = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    R = C = 0
    grid = []

    def in_bounds(self, r, c):
        return 0 <= r < self.R and 0 <= c < self.C
    
    def is_loop(self, r, c):
        vis = set()
        p = 0
        while self.in_bounds(r, c):
            if (r, c, p) in vis: return True
            vis.add((r, c, p))
            dr, dc = self.DIR[p]
            r += dr
            c += dc
            if self.in_bounds(r, c) and self.grid[r][c] == "#":
                r -= dr
                c -= dc
                p = (p + 1) % len(self.DIR)
        return False

    def calculate(self, filename):
        with open(filename, "r") as f:
            self.grid = [list(x) for x in f.read().splitlines()]
            self.R, self.C = len(self.grid), len(self.grid[0])
            vis = set()
            sr = sc = 0
            for r, c in product(range(self.R), range(self.C)):
                if self.grid[r][c] == "^":
                    sr, sc = r, c
                    break
            p = ans = 0
            while self.in_bounds(r, c):
                vis.add((r, c))
                dr, dc = self.DIR[p]
                r += dr
                c += dc
                if self.in_bounds(r, c) and self.grid[r][c] == "#":
                    r -= dr
                    c -= dc
                    p = (p + 1) % len(self.DIR)
            for r, c in vis:
                self.grid[r][c] = "#"
                ans += self.is_loop(sr, sc)
                self.grid[r][c] = "."
            print(ans)
PatrolGrid().calculate("big.txt")

1686


In [15]:
from sortedcontainers import SortedList
class SegmentTree:
    def __init__(self, n, neutral, func):
        self.func = func
        self.neutral = neutral
        self.size = 1
        self.n = n
        while self.size<n:
            self.size*=2
        self.nodes = [neutral for _ in range(self.size*2)] 

    def ascend(self, segment_idx):
        while segment_idx > 0:
            segment_idx -= 1
            segment_idx >>= 1
            left_segment_idx, right_segment_idx = 2*segment_idx + 1, 2*segment_idx + 2
            self.nodes[segment_idx] = self.func(self.nodes[left_segment_idx], self.nodes[right_segment_idx])
        
    def update(self, segment_idx, val):
        segment_idx += self.size - 1
        self.nodes[segment_idx] = val
        self.ascend(segment_idx)
            
    def query(self, left, right):
        stack = [(0, self.size, 0)]
        result = self.neutral
        while stack:
            # BOUNDS FOR CURRENT INTERVAL and idx for tree
            segment_left_bound, segment_right_bound, segment_idx = stack.pop()
            # NO OVERLAP
            if segment_left_bound >= right or segment_right_bound <= left: continue
            # COMPLETE OVERLAP
            if segment_left_bound >= left and segment_right_bound <= right:
                result = self.func(result, self.nodes[segment_idx])
                continue
            # PARTIAL OVERLAP
            mid_point = (segment_left_bound + segment_right_bound) >> 1
            left_segment_idx, right_segment_idx = 2*segment_idx + 1, 2*segment_idx + 2
            stack.extend([(mid_point, segment_right_bound, right_segment_idx), (segment_left_bound, mid_point, left_segment_idx)])
        return result
    
    def __repr__(self):
        return f"nodes array: {self.nodes}"
def calculate(filename):
    with open(filename, "r") as f:
        data = f.read()
        arr = list(map(int, data))
        N = sum(arr)
        pos = [-1] * N
        blocks_location = {}
        blocks = []
        updates = {}
        updated_blocks = set()
        size_map = {}
        free_space = [SortedList() for _ in range(N)]
        seg = SegmentTree(N, math.inf, min)
        j = 0
        for i in range(len(arr)):
            if i % 2 == 0:
                assert(arr[i] > 0)
                blocks_location[i // 2] = j
                for _ in range(arr[i]):
                    pos[j] = i // 2
                    j += 1
                blocks.append(arr[i])
            else:
                free_space[arr[i]].add(j)
                size_map[j] = arr[i]
                if len(free_space[arr[i]]) == 1: 
                    seg.update(arr[i], j)
                j += arr[i]
        # print(size_map)
        for i in range(len(blocks) - 1, 0, -1):
            p = seg.query(blocks[i], N + 1)
            # print(i, blocks[i], p)
            if p >= blocks_location[i]: continue
            assert(blocks_location[i] > p)
            free_space_size = size_map[p]
            free_space[free_space_size].pop(0)
            if len(free_space[free_space_size]) > 0:
                seg.update(free_space_size, free_space[free_space_size][0])
            else:
                seg.update(free_space_size, math.inf)
            rem_space = free_space_size - blocks[i]
            updates[p] = i
            p += blocks[i]
            if rem_space > 0:
                size_map[p] = rem_space
                free_space[rem_space].add(p)
                seg.update(rem_space, free_space[rem_space][0])
            updated_blocks.add(i)
        res = [-1] * N
        for k, v in updates.items():
            for i in range(blocks[v]):
                res[k + i] = v
        for i in range(N):
            if pos[i] == -1: continue
            if pos[i] not in updated_blocks:
                res[i] = pos[i]
        # j = N - 1
        # for i in range(N):
        #     if pos[i] == -1:
        #         while j > i and pos[j] == -1:
        #             j -= 1
        #         if j <= i: break
        #         pos[i], pos[j] = pos[j], pos[i]
        ans = 0
        for i in range(N):
            if res[i] == -1: continue
            ans += res[i] * i
        print(ans)

# calculate("small.txt")
calculate("big.txt")

6363268339304


In [ ]:
8592266602739

In [ ]:
from parse import compile

def calculate(filename):
    with open(filename, "r") as f:
        data = f.read().splitlines()
        row_translate = compile("rotate row y={:d} by {:d}")
        column_translate = compile("rotate column x={:d} by {:d}")
        rect = compile("rect {:d}x{:d}")
        ans = 0
        for line in data:
            print(line)
        print(ans)

# calculate("small.txt")
calculate("big.txt")

rect 1x1
rotate row y=0 by 20
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 4
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 5
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 6
rect 5x1
rotate row y=0 by 2
rect 1x3
rotate row y=2 by 8
rotate row y=0 by 8
rotate column x=0 by 1
rect 7x1
rotate row y=2 by 24
rotate row y=0 by 20
rotate column x=5 by 1
rotate column x=4 by 2
rotate column x=2 by 2
rotate column x=0 by 1
rect 7x1
rotate column x=34 by 2
rotate column x=22 by 1
rotate column x=15 by 1
rotate row y=2 by 18
rotate row y=0 by 12
rotate column x=8 by 2
rotate column x=7 by 1
rotate column x=5 by 2
rotate column x=2 by 1
rotate column x=0 by 1
rect 9x1
rotate row y=3 by 28
rotate row y=1 by 28
rotate row y=0 by 20
rotate col

In [ ]:
34, 1150